# ECS 171 Homework 4
## by, Jeffrey Ugochukwu
### 1a
Use the churn data set. Except for the Account Length, VMail Message, Day Mins, Day Calls, Eve Mins, Night Mins, Intl Mins, Intl Calls, CustServ Calls, Intl Plan, and Churn, drop the remaining columns from the dataset. Preprocess the data :
Encode categorical features to numerical values using one-hot encoding.
Use MinMaxScaler to transform your numerical attributes. One of the practices for training a Neural Network is to normalize your data to obtain a mean close to 0. Normalizing the data generally speeds up learning and leads to faster convergence.
Develop a 4-layer artificial neural network (ANN) and specifically a feed-forward multilayer perceptron (with sigmoid activations and MSE loss function) to perform binary classification to classify 'Churn' based on other variables. For this, split the data into training and testing set by 70:30 and use the training set for training the model and the test set to evaluate the model performance. Use sgd optimizer with no momentum and keep it simple. Please note that this is a binary classification problem so select the right number of nodes accordingly for the output layer. Use gridsearch to determine the number of nodes in the hidden layers. Train this toy model on a single iteration. Get the layer weights and loss from the model. You don’t need to use a callback since the current weights and loss will be available.

In [48]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.covariance import EllipticEnvelope
from sklearn.svm import OneClassSVM
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
import tensorflow as tf
from keras.layers import Dense
from keras.layers import Activation 
from tensorflow.keras import layers
from tensorflow.keras import initializers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
#Importing Churn dataset and dropping unnecessary columns
df = pd.read_csv("churn.txt")
df.drop(['State', 'Area Code', 'Phone', 'VMail Plan', 'Day Charge', 'Eve Calls', 'Eve Charge', 'Night Calls', 'Night Charge', 'Intl Charge'], axis=1, inplace=True)
df.head()

,Account Length,Int'l Plan,VMail Message,Day Mins,Day Calls,Eve Mins,Night Mins,Intl Mins,Intl Calls,CustServ Calls,Churn?
0,128,no,25,265.1,110,197.4,244.7,10.0,3,1,False.
1,107,no,26,161.6,123,195.5,254.4,13.7,3,1,False.
2,137,no,0,243.4,114,121.2,162.6,12.2,5,0,False.
3,84,yes,0,299.4,71,61.9,196.9,6.6,7,2,False.
4,75,yes,0,166.7,113,148.3,186.9,10.1,3,3,False.


In [50]:
#Organizing all of the numerical and categorical columns as separate varibales to be transformed and processed
#Also dropped the "Churn?_False." column since it's irrelevant since we need to classify that the values with churn existing
numerical_features = df.select_dtypes(['int64', 'float64']).columns.values
categorical_features = df.select_dtypes(['object']).columns.values
numerical_transform = MinMaxScaler(feature_range=(-1,1))
numerical_processed = numerical_transform.fit_transform(df[numerical_features])
df_processed = pd.get_dummies(df, columns= categorical_features)
df_processed.update(pd.DataFrame(numerical_processed, columns = numerical_features))
df_processed.drop(["Churn?_False."], axis = 1, inplace = True)
df_processed.head()

,Account Length,VMail Message,Day Mins,Day Calls,Eve Mins,Night Mins,Intl Mins,Intl Calls,CustServ Calls,Int'l Plan_no,Int'l Plan_yes,Churn?_True.
0,0.049587,-0.019608,0.511403,0.333333,0.085510,0.191501,0.00,-0.7,-0.777778,1,0,0
1,-0.123967,0.019608,-0.078677,0.490909,0.075062,0.243679,0.37,-0.7,-0.777778,1,0,0
2,0.123967,-1.000000,0.387685,0.381818,-0.333517,-0.250134,0.22,-0.5,-1.000000,1,0,0
3,-0.314050,-1.000000,0.706956,-0.139394,-0.659610,-0.065627,-0.34,-0.3,-0.555556,0,1,0
4,-0.388430,-1.000000,-0.049601,0.369697,-0.184493,-0.119419,0.01,-0.7,-0.333333,0,1,0


In [51]:
#Checking all of the names of the columns that currently exist
list(df_processed.columns)

['Account Length',
 'VMail Message',
 'Day Mins',
 'Day Calls',
 'Eve Mins',
 'Night Mins',
 'Intl Mins',
 'Intl Calls',
 'CustServ Calls',
 "Int'l Plan_no",
 "Int'l Plan_yes",
 'Churn?_True.']

In [52]:
#Building the ANN model
train, test = train_test_split(df_processed, train_size=0.7, random_state=1)
np.random.seed(1)
tf.random.set_seed(1)
def create_model(layer1=3, layer2=5):
    model = keras.Sequential()
    model.add(Dense(layer1, input_shape=(11,)))
    model.add(Activation('sigmoid')) 
    model.add(Dense(layer2))
    model.add(Activation('sigmoid'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy']) 
    return model
model = KerasClassifier(build_fn=create_model)
param_grid = {'layer1':[3, 5, 7], 'layer2':[3, 5]}
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(train.drop(columns=['Churn?_True.']), train['Churn?_True.'])
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

73/73 [==============================] - 1s 736us/step - loss: 0.3595 - accuracy: 0.1484
Best: 0.855115 using {'layer1': 3, 'layer2': 5}


### 1b
For the ANN in part a), calculate the first round of weight updates with back-propagation with paper and pencil for the two final layers (output to 2nd hidden layer; 2nd hidden layer to 1st hidden layer) for only the first sample. Limit this calculation to only the weights corresponding to (i) the single output node and (ii) a single hidden node from the 2nd hidden layer. These weights also include the bias. You can initialize all weights to zero except the weights that you are calculating, which can be initialized to 1. All nodes within the same layer will have the same weights and gradients. Confirm that the numbers you calculated are the same as those produced by the code and provide both your calculations and the code output. Make sure that the loss function used in your hand calculation is the same as the one used in Keras. If your calculations do not agree, find out why. Provide both calculations made by hand (scanned image and using a calculator/ computer to verify the results for each step is fine) and corresponding output from the program that shows that both are in agreement. Include all your assumptions in your answer. For this problem, it is sufficient to calculate a single incoming weight and the bias for each layer. This is because we are initializing all layers of the network to non-random, identical values (per layer). As a result the gradients will be the same across each layer.

In [25]:
#Fitting Logistic Regression model for ANN
mpg_x = np.array(df_processed['Account Length']).reshape(-1,1)
mpg_y = np.array(df_processed["Churn?_True."])
mpg_x_train,mpg_x_test,mpg_y_train,mpg_y_test = train_test_split(mpg_x,mpg_y,test_size=0.3)
mpg_log = LogisticRegression()
mpg_log.fit(mpg_x_train, mpg_y_train)
mpg_y_pred = mpg_log.predict(np.array(mpg_x_test))
mpg_y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [26]:
#Finding MSE for ANN
mean_squared_error(mpg_y_test, mpg_y_pred)

0.151

### 2
Create 2 dummy input samples and use the ANN model which you trained in Q1a to classify the output. Include your code and the classified output.

In [53]:
#Splitting the model using Train Test Split
xtrain = train.iloc[:, 0:11]
ytrain = train.iloc[:, 11:12]
xtest = test.iloc[:, 0:11]
ytest = test.iloc[:, 11:12]
model.fit(xtrain, ytrain, epochs = 350, verbose = 0)
xtrain.describe()

,Account Length,VMail Message,Day Mins,Day Calls,Eve Mins,Night Mins,Intl Mins,Intl Calls,CustServ Calls,Int'l Plan_no,Int'l Plan_yes
count,2333.000000,2333.000000,2333.000000,2333.000000,2333.000000,2333.000000,2333.000000,2333.000000,2333.000000,2333.000000,2333.000000
mean,-0.177798,-0.691469,0.029285,0.218299,0.104062,-0.043222,0.023759,-0.552679,-0.652522,0.907844,0.092156
std,0.331476,0.535288,0.309476,0.245939,0.277337,0.272998,0.277326,0.241756,0.293857,0.289308,0.289308
min,-1.000000,-1.000000,-1.000000,-1.000000,-0.766291,-1.000000,-1.000000,-1.000000,-1.000000,0.000000,0.000000
25%,-0.404959,-1.000000,-0.177309,0.054545,-0.088260,-0.224314,-0.150000,-0.700000,-0.777778,1.000000,0.000000
50%,-0.181818,-1.000000,0.034778,0.224242,0.100907,-0.044648,0.030000,-0.600000,-0.777778,1.000000,0.000000
75%,0.041322,-0.333333,0.236032,0.381818,0.297223,0.138784,0.200000,-0.400000,-0.555556,1.000000,0.000000
max,1.000000,1.000000,0.977195,1.000000,0.989552,1.000000,1.000000,0.900000,1.000000,1.000000,1.000000


In [57]:
#Created ANN Model
train, test = train_test_split(df_processed, train_size=0.7, random_state=1)
np.random.seed(1)
tf.random.set_seed(1)
def create_model(layer1=3, layer2=5):
    model = keras.Sequential()
    model.add(Dense(layer1, input_shape=(11,)))
    model.add(Activation('sigmoid')) 
    model.add(Dense(layer2))
    model.add(Activation('sigmoid'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy']) 
    return model
model = KerasClassifier(build_fn=create_model)
param_grid = {'layer1':[3, 5, 7], 'layer2':[3, 5]}
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(train.drop(columns=['Churn?_True.']), train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

73/73 [==============================] - 0s 895us/step - loss: 0.7461 - accuracy: 0.0960
Best: 0.613196 using {'layer1': 3, 'layer2': 5}


### 3
Change the hidden layer activation functions to ReLU, the output layer activation to softmax, and the loss function to cross-entropy. Is this a better choice?

In [46]:
#Building the ANN model
train, test = train_test_split(df_processed, train_size=0.7, random_state=1)
np.random.seed(1)
tf.random.set_seed(1)
def create_model(layer1=3, layer2=5):
    model = keras.Sequential()
    model.add(Dense(layer1, input_shape=(11,)))
    model.add(Activation(tf.nn.relu)) 
    model.add(Dense(layer2))
    model.add(Activation(tf.nn.relu))
    model.add(Dense(1))
    model.add(Activation('softmax'))
    model.compile(loss= 'binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
    return model
model = KerasClassifier(build_fn=create_model)
param_grid = {'layer1':[3, 5, 7], 'layer2':[3, 5]}
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(train.drop(columns=['Churn?_True.']), train['Churn?_True.'])
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

73/73 [==============================] - 1s 1ms/step - loss: 0.6404 - accuracy: 0.1484
Best: 0.144885 using {'layer1': 3, 'layer2': 3}


Changing the parameters to ReLU, Cross Entropy, and Softmax actually worsens the performance of the model where it has a best accuracy level of 0.144885, but the original ANN model had a best accuracy level of 0.855115. Therefore, changing the original paramters was a poor choice.

### 4
Describe the trade-off between a large hidden layer and a small one. What are the benefits, what are the drawbacks? Also, describe the benefits and drawbacks of using a large or small values for learning rate in a NN.

### Answer:
The tradeoffs using a large hidden layer in a neural network is that it decreases training training error but reduces generalization in classification. A small hidden would actually provide the inverse effect of what was explained for the tradeoffs in a large hidden layer. The tradeoffs in using a small learning rate for a neural network would be that this would require more epochs due to the small changes in weights per update, but it improves the convergence of a model to reach a specific values. The tradeoffs for a large learning rate be that it would require less eppchs, but it would make it harder for the model to converge to a value.